In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik
# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1600 * 128
RECS = 1384 * 1
TRANSL = 307

X = np.zeros([RECS, LENGTH, 1], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(1):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-pure-timedomain-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-pure-timedomain-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

Xpart.shape

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1377 1384


In [27]:
# set(np.where(X[0, :-1] == X[0, 1:])[0]) & set(np.where(X[0, :-1] == 0)[0])
len(X[0]) / 128

1600.0

In [28]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

from keras.constraints import max_norm

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

logger = MyCallback("./training-bigger-log.txt")
NFEATS = 1

shift = np.exp(2 * np.pi * 1j / 512)


class CZT(keras.layers.Layer):
    def build(self, input_shape):
        self.z = self.add_weight(shape=[2], initializer=lambda shape: K.variable([np.real(shift), np.imag(shift)]),
                                 name='kernel_z', constraint=max_norm(1.))
        self.w = self.add_weight(shape=[2], initializer=lambda shape: K.variable([1, 0]),
                                 name='kernel_w', constraint=max_norm(1.))
    
    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = 257
        return tuple(output_shape)
    
    def call(self, inputs):
        ints = K.reshape(K.arange(257, dtype='complex64'), [257, 1])
        k = K.reshape(K.arange(512, dtype='complex64'), [1, 512])
        z = K.cast(self.z[0], dtype='complex64_ref') + 1j * K.cast(self.z[1], dtype='complex64_ref')
        w = K.cast(self.w[0], dtype='complex64_ref') + 1j * K.cast(self.w[1], dtype='complex64_ref')
        weights = K.dot(z * K.ones([257, 1], dtype='complex64'), K.reshape(w, [1, -1]) ** (-k)) ** (-ints)
        print(z.shape, w.shape, weights.shape)
        czt = K.dot(K.cast(inputs, dtype='complex64_ref'), K.transpose(weights))
        return K.abs(czt)
    
def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    conv_layer = Conv1D(512, 512, strides=128, activation='relu', bias=False)
    conv_layer.build((None, NFEATS))
    conv_layer.trainable = False
    conv_layer.set_weights([np.eye(512, dtype=np.float32).reshape([512, 1, 512])])
    conv_layer = conv_layer(feature_input)
    layer = CZT()(conv_layer)
    layer = Lambda(lambda x: (x - MEAN) / STD)(layer)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    for i in range(7):
        layer = Bidirectional(GRU(256, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear'))(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
    layer = Dense(1024)(layer)
    layer = LeakyReLU(0.01)(layer)
    layer = BatchNormalization()(layer)
    layer_15 = Dense(NPHONES + 1, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = 100,
                     batch_size = 8, callbacks=[mc, logger])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.array([400 for x in X_lens], np.int16)# np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=1000) # at 300 it makes sensible predictions
    valid_data = make_data(*load_data(VALIDDATA))
    predictions = predict.predict(valid_data[0])
    DERs = validate(predictions, valid_data[2], valid_data[1], valid_data[3])
    print("Validation Digit Error Rate: {}".format(sum([x[0] * x[1] for x in DERs]) / sum([x[1] for x in DERs])))
    predict.save('cyfry.pred.h5')

[(1376, 204800, 1), (1376, 307), (1376, 1), (1376, 1)]


/venv/lib/python3.5/site-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(512, 512, use_bias=False, activation="relu", strides=128)`


() () (257, 512)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, None, 512)    262144      input_3[0][0]                    
__________________________________________________________________________________________________
czt_2 (CZT)                     (None, None, 257)    4           conv1d_3[0][0]                   
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, None, 257)    0           czt_2[0][0]                      
____________________________________________________________________________________________

Epoch 1/100
1376/1376 [==============================] - 896s 651ms/step - loss: 348.1899
Epoch 2/100
1376/1376 [==============================] - 883s 642ms/step - loss: 269.9879
Epoch 3/100
1376/1376 [==============================] - 884s 642ms/step - loss: 267.1382
Epoch 4/100
1376/1376 [==============================] - 884s 642ms/step - loss: 266.1219
Epoch 5/100
1376/1376 [==============================] - 883s 642ms/step - loss: 265.4635
Epoch 6/100
1376/1376 [==============================] - 883s 642ms/step - loss: 263.9082
Epoch 7/100
1376/1376 [==============================] - 884s 642ms/step - loss: 262.5855
Epoch 8/100
1376/1376 [==============================] - 883s 641ms/step - loss: nan
Epoch 9/100
1376/1376 [==============================] - 881s 640ms/step - loss: nan
Epoch 10/100
1376/1376 [==============================] - 881s 640ms/step - loss: nan
Epoch 11/100
1376/1376 [==============================] - 881s 641ms/step - loss: nan
Epoch 12/100
1376/1376 [====

KeyboardInterrupt: 

In [7]:
pr = predict.predict(X[:1])
pr = pr.argmax(2)[0]
pr

array([ 5,  5, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
       37, 37,  8, 37, 37, 37, 32, 32, 37, 12, 37, 37, 36, 37, 37, 32, 37,
       12, 37,  8, 37, 37, 33, 37, 37, 19, 37, 23, 37, 20, 20, 37, 19, 37,
       37, 31, 37, 32, 32, 37, 36, 37,  7, 25, 25, 16, 16, 37, 36, 37, 13,
       37, 31, 31,  2, 37, 31, 37, 37, 37, 19, 37, 23, 23, 37, 20, 37, 37,
       12, 37, 25, 37, 37, 23, 23, 37, 37, 17, 17, 37, 20, 37, 37,  5,  5,
        5, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  8, 37, 37,
       32, 37, 37, 32, 37, 37, 36, 36, 37, 37, 32, 37, 37, 20, 37,  2, 37,
       37, 37, 32, 37, 37,  3, 37, 37,  8, 37, 37,  0, 37, 37, 23, 37, 37,
       37, 37, 22, 37, 36, 37, 37,  4,  4, 37, 37, 24, 37, 20, 37, 32, 12,
       12, 12, 37, 33, 33

In [12]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
pr.eval(session=K.get_session())

array([[ 5,  8, 32, 12, 36, 32, 12,  8, 33, 19, 23, 20, 19, 31, 32, 36,
         7, 25, 16, 36, 13, 31,  2, 31, 19, 23, 20, 12, 25, 23, 17, 20,
         5,  8, 32, 32, 36, 32, 20,  2, 32,  3,  8,  0, 23, 22, 36,  4,
        24, 20, 32, 12, 33, 20, 12,  8, 18,  5]])

In [13]:
Y[:1]

array([[ 5,  3,  8, 32, 12, 36,  5, 32, 19,  8, 33, 31, 19, 23, 20,  5,
        19, 31, 32, 28,  7, 25, 16, 36, 13, 31,  2,  5, 31, 19,  5, 23,
        20, 19, 25,  5, 23, 17, 20,  5,  8,  5, 32, 32, 36, 32, 20,  5,
         2,  6,  3, 36, 17, 23,  5, 22, 36,  4, 24, 20,  5, 32, 12, 33,
        20, 12,  5,  8, 18,  5, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [20]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

0.2571428571428571

In [22]:
pr = predict.predict(X[:160])
pr = K.ctc_decode(pr, X_lens[:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.2571428571428571,
 0.32558139534883723,
 0.43548387096774194,
 0.4125,
 0.48760330578512395,
 0.3617021276595745,
 0.25925925925925924,
 0.24444444444444444,
 0.27941176470588236,
 0.41739130434782606,
 0.35443037974683544,
 0.3888888888888889,
 0.3230769230769231,
 0.4411764705882353,
 0.4358974358974359,
 0.4264705882352941,
 0.3026315789473684,
 0.4146341463414634,
 0.39325842696629215,
 0.3855421686746988,
 0.39285714285714285,
 0.3488372093023256,
 0.35051546391752575,
 0.3627450980392157,
 0.28846153846153844,
 0.32673267326732675,
 0.23376623376623376,
 0.4090909090909091,
 0.3472222222222222,
 0.37373737373737376,
 0.30337078651685395,
 0.4027777777777778,
 0.3333333333333333,
 0.373134328358209,
 0.34375,
 0.3974358974358974,
 0.3132530120481928,
 0.35443037974683544,
 0.4069767441860465,
 0.3448275862068966,
 0.3333333333333333,
 0.32142857142857145,
 0.3253012048192771,
 0.3103448275862069,
 0.4835164835164835,
 0.2826086956521739,
 0.36619718309859156,
 0.278481012658227

In [24]:
np.array(_).mean()

0.36606030307985016

In [28]:
predict.layers[1].z.eval(session=K.get_session())

array([0.99589145, 0.00921931], dtype=float32)

In [29]:
predict.layers[1].w.eval(session=K.get_session())

array([0.99997854, 0.00653262], dtype=float32)